# Ideas for solving the viscous profile

We are considering the problem
$$
\partial_t q + \partial_x f(q) = \kappa \partial_{xx} q
$$
where $\kappa > 0$. We want to look for a viscous, or travelling wave, profile, where the solution is given by
$$
q(t, x) \equiv Q \left( \frac{x - V t}{\kappa} \right) \equiv Q(\xi).
$$
This converts the equation of motion to the second order ODE
$$
\left( \partial_q f - V \right) Q' = Q''.
$$
We also need to impose the boundary conditions
$$
\begin{aligned}
  \lim_{\xi \to -\infty} Q(\xi) & = q_L, \\
  \lim_{\xi \to +\infty} Q(\xi) & = q_R, \\
  \lim_{|\xi| \to \infty} Q'(\xi) & = 0.
\end{aligned}
$$
With these boundary conditions the solution in the vanishing viscosity limit $\kappa \to 0_+$ will give the Rankine-Hugoniot conditions.

At this stage the problem is not well-posed. We have a second order ODE with a single unknown parameter $V$, the speed of the travelling wave. However, we have four boundary conditions. Generalising to a system of size $n$, we will have $2n$ integration constants plus the value of $V$ that need fixing and $4n$ boundary conditions.

## Steps

As a first step we note that the ODE is unchanged if we translate it, $\xi \to \xi + A$. We can therefore impose one additional condition in order to fix this translation constant. For example, we can impose
$$
Q(0) = (q_L + q_R) / 2.
$$
Intuitively this "centres" the profile within the domain.

As a second step we can change coordinates via a compactification. This should lead to a singular boundary value problem and hence incorporate the derivative boundary conditions in automatically.

Explicitly, introduce
$$
z_{\pm} = \pm \frac{\xi}{1 \pm \xi}.
$$
We have that $\xi = 0 \iff z_{\pm} = 0$, $\xi \to +\infty \iff z_+ \to 1$, and $\xi \to -\infty \iff z_- \to 1$. Therefore both $z_{\pm} \in [0, 1)$, and together they cover the full range of $\xi$.

Note that
$$
\xi = \pm \frac{z_{\pm}}{1 - z_{\pm}}.
$$

We now change variable. Write (for example) $Q = Q(z_+)$. We have
$$
\partial_\xi = \frac{\partial z_+}{\partial \xi} \partial_{z_+} = \frac{1}{(1+\xi)^2} \partial_{z_+} = (1 - z_+)^2 \partial_{z_+}.
$$
This means the ODE becomes, for $\xi \in [0, \infty) \iff z_+ \in [0, 1)$,
$$
\left( \partial_q f - V + 2 (1 - z_+) \right) Q' = (1 - z_+)^2 Q'',
$$
where the primes should now be interpreted as derivatives with respect to $z_+$. Similary, the ODE becomes, for $\xi \in (-\infty, 0] \iff z_- \in [0, 1)$,
$$
\left( \partial_q f - V - 2 (1 - z_-) \right) Q' = -(1 - z_-)^2 Q'',
$$
where the primes should now be interpreted as derivatives with respect to $z_-$.

We now write (with some abuse of notation)
$$
Q = \begin{cases} Q_+(z_+) & \xi > 0, \\ Q_-(z_-) & \xi < 0. \end{cases}
$$
The system of ODEs now becomes, in first order form,
$$
\begin{aligned}
  Q_+' &= R_+ \\
  Q_-' &= R_- \\
  (1 - z)^2 R_+' &= \left( \partial_q f - V + 2 ( 1 - z ) \right) R_+ \\
  -(1 - z)^2 R_-' &= \left( \partial_q f - V - 2 ( 1 - z ) \right) R_-.
\end{aligned}
$$
All primes are derivatives with respect to $z$; we have identified $z_{\pm}$ with $z$ as the ranges match. The boundary conditions are
$$
\begin{aligned}
  Q_+(0) &= (q_L + q_R) / 2, \\
  Q_+(1) &= q_R, \\
  Q_-(1) &= q_L, \\
  Q_-(0) &= Q_+(0), \\
  R_-(0) &= -R_+(0).
\end{aligned}
$$
That is, we are fixing the translation constant by enforcing the value at $z=0$, we are imposing the function values at $\xi = \pm \infty$, and we enforcing the continuity of the function and its first derivative at the matching point $\xi = 0 = z_\pm$.

This gives us an ODE system of size four with one additional unknown parameter, plus five boundary conditions. This should (?) be a well-posed problem.

### Singular BVPs

We note that the BVP that we actually have to solve is going to be singular, as it has the form
$$
R' = \frac{1}{1 - z^2} {\cal S}(Q, R; V).
$$

The `scipy` function `solve_bvp` can enforce singular boundary conditions, but only at the *left* boundary. Ours is at the right. So we need to make another change of variable; probably something like $Z = 1 - z$, which leads to a system something like (**needs checking**)
$$
\begin{aligned}
  Q_+' &= -R_+ \\
  Q_-' &= -R_- \\
  -Z^2 R_+' &= \left( \partial_q f - V + 2 Z \right) R_+ \\
  Z^2 R_-' &= \left( \partial_q f - V - 2 Z \right) R_-.
\end{aligned}
$$
We also need to flip the boundary condition locations $0 \leftrightarrow 1$. The singularity is now at $Z=0$ and the matching point at $Z=1$.

In [3]:
### Coding it Up

In [5]:
import numpy as np
import sympy
sympy.init_printing()
from scipy.optimize import brentq, newton, minimize_scalar
from scipy.integrate import solve_ivp, solve_bvp
from scipy.special import erf
import matplotlib.pyplot as plt

In [6]:
Z = sympy.symbols('Z')
V = sympy.symbols('V')
Qp = sympy.Function('Qp')
Qm = sympy.Function('Qm')
Rp = sympy.Function('Rp')
Rm = sympy.Function('Rm')

In [15]:
def f_bvp(eta, q, p):
    V = p[0]
    Qp, Qm, Rp, Rm = q
    f = np.zeros_like(q)
    f[0, :] = -Rp
    f[1, :] = -Rm
    f[2, :] = -(1/Z**2)*(2*Qp - V + 2*Z)*Rp
    f[3, :] = (1/Z**2)*(2*Qm - V - 2*Z)*Rm
    return f
def bcs(qa, qb, p):
    return np.array([qa[0] + 1, qb[0], qa[1] - 1, qb[1], qb[2] + qb[3]])

In [21]:
Z = np.linspace(0, 1)
q_guess = np.zeros((4, len(Z)))
q_guess[0, :] = - Z / 100
q_guess[1, :] = Z / 100
q_guess[2, :] = - 1 / 100
q_guess[3, :] = 1 / 100

S = np.zeros((4,4))
S[0,2] = (1/Z**2)
S

<ipython-input-21-a193e6f4dbdc>:9: RuntimeWarning: divide by zero encountered in true_divide
  S[0,2] = (1/Z**2)


ValueError: setting an array element with a sequence.

In [17]:

soln = solve_bvp(f_bvp, bcs, Z, q_guess, p = [0], tol=5e-2, S)
#soln = solve_bvp(f_bvp, bcs, soln.x, soln.y, p = [0], tol=1e-2)
soln.success

<ipython-input-15-81519e3b2c72>:7: RuntimeWarning: divide by zero encountered in true_divide
  f[2, :] = -(1/Z**2)*(2*Qp - V + 2*Z)*Rp
<ipython-input-15-81519e3b2c72>:7: RuntimeWarning: invalid value encountered in multiply
  f[2, :] = -(1/Z**2)*(2*Qp - V + 2*Z)*Rp
<ipython-input-15-81519e3b2c72>:8: RuntimeWarning: divide by zero encountered in true_divide
  f[3, :] = (1/Z**2)*(2*Qm - V - 2*Z)*Rm
<ipython-input-15-81519e3b2c72>:8: RuntimeWarning: invalid value encountered in multiply
  f[3, :] = (1/Z**2)*(2*Qm - V - 2*Z)*Rm


ValueError: operands could not be broadcast together with shapes (49,) (50,) 